In [2]:
# Initial imports
import pandas as pd
from sklearn.cluster import KMeans
import plotly.express as px
import hvplot.pandas

In [76]:
# Read the CSV file into a Pandas DataFrame

districts = pd.read_csv('District_data.csv')
districts = districts.replace('redacted',0)
districts.head()

,District,County,District_type,Enrollment,FRL_Perc,Disadv_Perc,EL_Perc,Grad_Perc,Teach_to_stud,Susp_Perc,Chronic_absent,Math_metAbove,ELA_metAbove,Per_pupil_exp,Teacher_salary,Avg Years Teaching (District)
0,Happy Camp Union Elementary (Siskiyou),Siskiyou,Elementary School District,110,77.3,77.27,NaN,NaN,NaN,7.9,29.5,9.23,17.91,13585,76081,NaN
1,Shoreline Unified (Marin),Marin,Unified School District,508,66.9,68.90,42.1,94.3,NaN,3.7,17.7,27.41,43.63,29742,87808,NaN
2,Cienega Union Elementary (San Benito),San Benito,Elementary School District,25,32.0,44.00,28.0,NaN,0.0,0,6.3,35.00,42.11,11515,76081,NaN
3,Alpine County Office of Education (Alpine),Alpine,County Office of Education (COE),6023,0.0,0.00,NaN,NaN,0.0,NaN,NaN,37.20,48.30,14708,76081,NaN
4,Arena Union Elementary/Point Arena Joint Union...,Mendocino,Common Administration District,6023,56.8,60.00,18.4,NaN,20.2,3.5,13.1,37.30,48.30,22151,57730,NaN


In [77]:
# Rename average years of teaching column.
districts = districts.rename(columns={"Avg Years Teaching (District)": "Avg_years_teaching"})
districts

,District,County,District_type,Enrollment,FRL_Perc,Disadv_Perc,EL_Perc,Grad_Perc,Teach_to_stud,Susp_Perc,Chronic_absent,Math_metAbove,ELA_metAbove,Per_pupil_exp,Teacher_salary,Avg_years_teaching
0,Happy Camp Union Elementary (Siskiyou),Siskiyou,Elementary School District,110,77.3,77.27,NaN,NaN,NaN,7.9,29.5,9.23,17.91,13585,76081,NaN
1,Shoreline Unified (Marin),Marin,Unified School District,508,66.9,68.90,42.1,94.3,NaN,3.7,17.7,27.41,43.63,29742,87808,NaN
2,Cienega Union Elementary (San Benito),San Benito,Elementary School District,25,32.0,44.00,28.0,NaN,0.0,0,6.3,35.00,42.11,11515,76081,NaN
3,Alpine County Office of Education (Alpine),Alpine,County Office of Education (COE),6023,0.0,0.00,NaN,NaN,0.0,NaN,NaN,37.20,48.30,14708,76081,NaN
4,Arena Union Elementary/Point Arena Joint Union...,Mendocino,Common Administration District,6023,56.8,60.00,18.4,NaN,20.2,3.5,13.1,37.30,48.30,22151,57730,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,Lincoln Elementary (Marin),Marin,Elementary School District,4,0.0,0.00,NaN,NaN,0.0,0,0,37.20,48.30,86414,76081,1.0
1032,SBE - KIPP Bayview Elementary (San Francisco),San Francisco,State Board of Education Charter,140,90.0,90.00,2.1,NaN,35.0,7.4,37.1,37.20,48.30,14708,76081,1.0
1033,SBE - KIPP Navigate College Prep (Santa Clara),Santa Clara,State Board of Education Charter,178,75.3,82.02,14.6,NaN,22.3,7.1,11.3,37.20,48.30,14708,76081,1.0
1034,Forks of Salmon Elementary (Siskiyou),Siskiyou,Elementary School District,9,100.0,100.00,18.4,NaN,9.0,3.5,13.1,37.30,48.30,31008,76081,1.0


In [78]:
# Replace NaaNs with means.
districts = districts.fillna(districts.mean())

In [79]:
# Rename columns.
districts = districts.rename(columns={"Avg Years Teaching (District)": "Avg_years_teaching"})

In [80]:
# Replace NaaNs with means.
districts = districts.fillna(districts.mean())

In [81]:
districts.dtypes

District               object
County                 object
District_type          object
Enrollment              int64
FRL_Perc              float64
Disadv_Perc           float64
EL_Perc               float64
Grad_Perc             float64
Teach_to_stud         float64
Susp_Perc              object
Chronic_absent         object
Math_metAbove         float64
ELA_metAbove          float64
Per_pupil_exp           int64
Teacher_salary          int64
Avg_years_teaching    float64
dtype: object

In [82]:
# Create new dataframe with only continuous columns included.
dist_cluster = districts
dist_cluster = dist_cluster.drop(['District','County','District_type','Susp_Perc','Chronic_absent'], axis=1)
dist_cluster

,Enrollment,FRL_Perc,Disadv_Perc,EL_Perc,Grad_Perc,Teach_to_stud,Math_metAbove,ELA_metAbove,Per_pupil_exp,Teacher_salary,Avg_years_teaching
0,110,77.3,77.27,18.437052,84.5431,20.193016,9.23,17.91,13585,76081,9.894221
1,508,66.9,68.90,42.100000,94.3000,20.193016,27.41,43.63,29742,87808,9.894221
2,25,32.0,44.00,28.000000,84.5431,0.000000,35.00,42.11,11515,76081,9.894221
3,6023,0.0,0.00,18.437052,84.5431,0.000000,37.20,48.30,14708,76081,9.894221
4,6023,56.8,60.00,18.400000,84.5431,20.200000,37.30,48.30,22151,57730,9.894221
...,...,...,...,...,...,...,...,...,...,...,...
1031,4,0.0,0.00,18.437052,84.5431,0.000000,37.20,48.30,86414,76081,1.000000
1032,140,90.0,90.00,2.100000,84.5431,35.000000,37.20,48.30,14708,76081,1.000000
1033,178,75.3,82.02,14.600000,84.5431,22.300000,37.20,48.30,14708,76081,1.000000
1034,9,100.0,100.00,18.400000,84.5431,9.000000,37.30,48.30,31008,76081,1.000000


# Create Elbow Curve

In [83]:
# Create an empty list to hold inertia values.
inertia = []
k = list(range(1, 11))

In [84]:
# Loop through, looking for the best K value.
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(dist_cluster)
    inertia.append(km.inertia_)

In [85]:
# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

# K Means Model

In [89]:
dist_cluster.hvplot.scatter(x="FRL_Perc", y="Per_pupil_exp")

:Scatter   [FRL_Perc]   (Per_pupil_exp)

In [90]:
# Function to cluster and plot dataset
def test_cluster_amount(dist_cluster, clusters):
    model = KMeans(n_clusters=clusters, random_state=5)
    model

    # Fitting model
    model.fit(dist_cluster)

    # Add a new class column to districts
    dist_cluster["class"] = model.labels_

In [91]:
# Run the clustering function to show clusters.
test_cluster_amount(dist_cluster, 3)
dist_cluster.hvplot.scatter(x="FRL_Perc", y="Per_pupil_exp", by="class")

:NdOverlay   [class]
   :Scatter   [FRL_Perc]   (Per_pupil_exp)

In [92]:
# 3D Plot.
fig = px.scatter_3d(
    dist_cluster,
    x="FRL_Perc",
    y="Per_pupil_exp",
    z="Enrollment",
    color="class",
    symbol="class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()